In [25]:
# Sys path
from sys import path
from pathlib import Path

module_path = str(Path.cwd().parents[1])

if module_path not in path:
    path.append(module_path)
    
path.append(module_path + '\\functions')


# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import data_preparation, mrmr
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import time
import pickle

---------

# Datasets

In [27]:
# FDI
df_fdi, X_fdi, y_fdi = data_preparation.load_dataset(module_path + '\\dataset\\multi_attack_FDI.csv')
X_fdi.head()

,U1 - SCADA,U2 - SCADA,U3 - SCADA,U4 - SCADA,U5 - SCADA,U6 - SCADA,U7 - SCADA,U8 - SCADA,U9 - SCADA,U10 - SCADA,U11 - SCADA,U12 - SCADA,U13 - SCADA,U14 - SCADA,P2 - SCADA,P3 - SCADA,P4 - SCADA,P5 - SCADA,P6 - SCADA,P7 - SCADA,P8 - SCADA,P9 - SCADA,P10 - SCADA,P11 - SCADA,P12 - SCADA,P13 - SCADA,P14 - SCADA,Q2 - SCADA,Q3 - SCADA,Q4 - SCADA,Q5 - SCADA,Q6 - SCADA,Q7 - SCADA,Q8 - SCADA,Q9 - SCADA,Q10 - SCADA,Q11 - SCADA,Q12 - SCADA,Q13 - SCADA,Q14 - SCADA,U1 - EM - EKF,U2 - EM - EKF,U3 - EM - EKF,U4 - EM - EKF,U5 - EM - EKF,U6 - EM - EKF,U7 - EM - EKF,U8 - EM - EKF,U9 - EM - EKF,U10 - EM - EKF,U11 - EM - EKF,U12 - EM - EKF,U13 - EM - EKF,U14 - EM - EKF,P2 - EM - EKF,P3 - EM - EKF,P4 - EM - EKF,P5 - EM - EKF,P6 - EM - EKF,P7 - EM - EKF,P8 - EM - EKF,P9 - EM - EKF,P10 - EM - EKF,P11 - EM - EKF,P12 - EM - EKF,P13 - EM - EKF,P14 - EM - EKF,Q2 - EM - EKF,Q3 - EM - EKF,Q4 - EM - EKF,Q5 - EM - EKF,Q6 - EM - EKF,Q7 - EM - EKF,Q8 - EM - EKF,Q9 - EM - EKF,Q10 - EM - EKF,Q11 - EM - EKF,Q12 - EM - EKF,Q13 - EM - EKF,Q14 - EM - EKF,U1 - FM - EKF,U2 - FM - EKF,U3 - FM - EKF,U4 - FM - EKF,U5 - FM - EKF,U6 - FM - EKF,U7 - FM - EKF,U8 - FM - EKF,U9 - FM - EKF,U10 - FM - EKF,U11 - FM - EKF,U12 - FM - EKF,U13 - FM - EKF,U14 - FM - EKF,P2 - FM - EKF,P3 - FM - EKF,P4 - FM - EKF,P5 - FM - EKF,P6 - FM - EKF,P7 - FM - EKF,P8 - FM - EKF,P9 - FM - EKF,P10 - FM - EKF,P11 - FM - EKF,P12 - FM - EKF,P13 - FM - EKF,P14 - FM - EKF,Q2 - FM - EKF,Q3 - FM - EKF,Q4 - FM - EKF,Q5 - FM - EKF,Q6 - FM - EKF,Q7 - FM - EKF,Q8 - FM - EKF,Q9 - FM - EKF,Q10 - FM - EKF,Q11 - FM - EKF,Q12 - FM - EKF,Q13 - FM - EKF,Q14 - FM - EKF,U1 - ES - EKF,U2 - ES - EKF,U3 - ES - EKF,U4 - ES - EKF,U5 - ES - EKF,U6 - ES - EKF,U7 - ES - EKF,U8 - ES - EKF,U9 - ES - EKF,U10 - ES - EKF,U11 - ES - EKF,U12 - ES - EKF,U13 - ES - EKF,U14 - ES - EKF,ph2 - ES - EKF,ph3 - ES - EKF,ph4 - ES - EKF,ph5 - ES - EKF,ph6 - ES - EKF,ph7 - ES - EKF,ph8 - ES - EKF,ph9 - ES - EKF,ph10 - ES - EKF,ph11 - ES - EKF,ph12 - ES - EKF,ph13 - ES - EKF,ph14 - ES - EKF,U1 - FS - EKF,U2 - FS - EKF,U3 - FS - EKF,U4 - FS - EKF,U5 - FS - EKF,U6 - FS - EKF,U7 - FS - EKF,U8 - FS - EKF,U9 - FS - EKF,U10 - FS - EKF,U11 - FS - EKF,U12 - FS - EKF,U13 - FS - EKF,U14 - FS - EKF,ph2 - FS - EKF,ph3 - FS - EKF,ph4 - FS - EKF,ph5 - FS - EKF,ph6 - FS - EKF,ph7 - FS - EKF,ph8 - FS - EKF,ph9 - FS - EKF,ph10 - FS - EKF,ph11 - FS - EKF,ph12 - FS - EKF,ph13 - FS - EKF,ph14 - FS - EKF,U1 - NI,U2 - NI,U3 - NI,U4 - NI,U5 - NI,U6 - NI,U7 - NI,U8 - NI,U9 - NI,U10 - NI,U11 - NI,U12 - NI,U13 - NI,U14 - NI,P2 - NI,P3 - NI,P4 - NI,P5 - NI,P6 - NI,P7 - NI,P8 - NI,P9 - NI,P10 - NI,P11 - NI,P12 - NI,P13 - NI,P14 - NI,Q2 - NI,Q3 - NI,Q4 - NI,Q5 - NI,Q6 - NI,Q7 - NI,Q8 - NI,Q9 - NI,Q10 - NI,Q11 - NI,Q12 - NI,Q13 - NI,Q14 - NI
0,1.059551,1.105877,1.024849,1.025205,1.023384,1.058997,1.048078,1.037532,1.054824,1.050296,1.050457,1.056671,1.049559,1.035974,0.754277,-0.635725,-0.396245,-0.160307,-0.064446,1.870000e-09,2.080000e-07,-0.169695,-0.051857,-0.020201,-0.035365,-0.078347,-0.086118,2.057798,-0.274881,-0.286298,-0.324909,-0.103359,1.720000e-11,-0.051262,0.115717,-0.033408,-0.010405,-0.009237,-0.033421,-0.029081,1.056849,1.102182,1.014983,1.019508,1.021200,1.054115,1.043560,1.035171,1.049691,1.046160,1.048129,1.045817,1.043318,1.036608,0.780019,-0.743372,-0.426913,-0.160683,-0.067163,0.000199,0.000173,-0.162646,-0.051450,-0.020009,-0.035432,-0.078589,-0.085432,2.118962,-0.274198,-0.289456,-0.323740,-0.104960,0.000766,-0.049302,0.114310,-0.034008,-0.010093,-0.009146,-0.033214,-0.028976,1.059842,1.045142,1.023106,1.024949,1.026078,1.059853,1.048921,1.040212,1.055245,1.051773,1.053777,1.051576,1.049064,1.042260,0.116914,-0.543501,-0.276172,-0.043833,-0.064897,0.000007,0.000003,-0.170477,-0.052066,-0.020105,-0.035227,-0.078040,-0.086155,0.047734,0.012437,0.022423,-0.009242,-0.103061,-0.000004,-0.051430,0.115839,-0.033496,-0.010416,-0.009271,-0.033568,-0.028907,1.056849,1.102182,1.014983,1.019508,1.021200,1.054115,1.043560,1.035171,1.049691,1.046160,1.048129,1.045

In [8]:
X_fdi.shape, y_fdi.shape

((3735, 214), (3735,))

------------

# FDI ML models

### 1. All features

In [13]:
X_train, X_test, y_train, y_test = data_preparation.split_data(X_fdi, y_fdi, test_size=0.2, random_state=123)

train = 1

if train==1:
    # training
    train_LR = -time.time()
    lr_train = LogisticRegression(random_state=0)
    lr_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\lr_model.pickle', 'wb') as f:
        pickle.dump(lr_train, f)
    train_LR += time.time()
    print('LR')

    train_KNN = -time.time()
    knn_train = KNeighborsClassifier(n_neighbors=3,
                               weights='distance',
                               )
    knn_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\knn_model.pickle', 'wb') as f:
        pickle.dump(knn_train, f)
    train_KNN += time.time()
    print('KNN')

    train_RF = -time.time()
    rf_train = RandomForestClassifier(n_estimators=600,
                                max_depth=14,
                                min_samples_leaf=4, 
                                min_samples_split=4,
                                random_state=0,
                                n_jobs=-1,
                                )
    rf_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\rf_model.pickle', 'wb') as f:
        pickle.dump(rf_train, f)
    train_RF += time.time()
    print('RF')
    
    train_XGB = -time.time()
    xgb_train = XGBClassifier(n_estimators=400,
                        max_depth=15,
                        eta=0.3,
                        subsample=0.9,
                        colsample_bytree=0.5, 
                        random_state=0,
                        n_jobs= -1,
                        silent=True)
    xgb_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\xgb_model.pickle', 'wb') as f:
        pickle.dump(xgb_train, f)
    train_XGB += time.time()
    print('XGB')
    
    train_time = data_preparation.to_dict(train_LR, train_KNN, train_RF, train_XGB)
    data_preparation.save_model(train_time, 'time/train_time')
    
train_time = data_preparation.load_model('time/train_time')
    
# load models
lr = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\lr_model.pickle', 'rb'))
knn = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\knn_model.pickle', 'rb'))
rf = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\rf_model.pickle', 'rb'))
xgb = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\xgb_model.pickle', 'rb'))
    

# prediction
test_LR = -time.time()
y_pred_lr = lr.predict(X_test)
test_LR += time.time()
    
test_KNN = -time.time()
y_pred_knn = knn.predict(X_test)
test_KNN += time.time()
    
test_RF = -time.time()
y_pred_rf = rf.predict(X_test)
test_RF += time.time()
    
test_XGB = -time.time()
y_pred_xgb = xgb.predict(X_test)
test_XGB += time.time()
       
# F1 score
macro_f1_lr = f1_score(y_test, y_pred_lr, average='macro')
macro_f1_knn = f1_score(y_test, y_pred_knn, average='macro')
macro_f1_rf = f1_score(y_test, y_pred_rf, average='macro')
macro_f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

LR
KNN
RF
XGB


In [15]:
print('##############')
print('F1 score:')
print('LR', macro_f1_lr*100) 
print('KNN', macro_f1_knn*100)
print('RF', macro_f1_rf*100)
print('XGB', macro_f1_xgb*100)

print('##############')
print('Training Time:')
print('LR', train_time['LR'], 'sec')
print('KNN', train_time['KNN'], 'sec')
print('RF', train_time['RF'], 'sec')
print('XGB', train_time['XGB'], 'sec')

print('##############')
print('Testing Time:')
print('LR', test_LR, 'sec')
print('KNN', test_KNN, 'sec')
print('RF', test_RF, 'sec')
print('XGB', test_XGB, 'sec')

##############
F1 score:
LR 50.42246876799985
KNN 27.533001398895436
RF 72.64705508391282
XGB 44.15602412751851
##############
Training Time:
LR 19.696067333221436 sec
KNN 0.03120875358581543 sec
RF 79.80140256881714 sec
XGB 942.3405983448029 sec
##############
Testing Time:
LR 0.03399968147277832 sec
KNN 0.3809998035430908 sec
RF 3.881359815597534 sec
XGB 51.24109101295471 sec
##############
Parameters:


------------

# MRMR

In [16]:
selected_features_fdi = mrmr.mrmr_classif(X_fdi, y_fdi, K = 100 , relevance = 'mi', redundancy = 'spearman', denominator = 'mean')

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 211.42it/s]


### 2. Best N features

In [20]:
N = 70
best_features = selected_features_fdi[:N]
best_features

['P13 - EM - EKF',
 'Q13 - EM - EKF',
 'Q6 - EM - EKF',
 'P6 - EM - EKF',
 'P12 - EM - EKF',
 'Q12 - EM - EKF',
 'P5 - EM - EKF',
 'Q9 - EM - EKF',
 'P4 - EM - EKF',
 'Q4 - EM - EKF',
 'Q7 - EM - EKF',
 'Q5 - EM - EKF',
 'P7 - EM - EKF',
 'P9 - EM - EKF',
 'P5 - SCADA',
 'P11 - EM - EKF',
 'Q14 - EM - EKF',
 'Q3 - EM - EKF',
 'P2 - EM - EKF',
 'Q10 - EM - EKF',
 'Q8 - EM - EKF',
 'Q4 - SCADA',
 'P14 - EM - EKF',
 'P4 - NI',
 'Q11 - EM - EKF',
 'P13 - SCADA',
 'P4 - SCADA',
 'P6 - SCADA',
 'Q2 - EM - EKF',
 'Q13 - SCADA',
 'P5 - NI',
 'Q6 - SCADA',
 'U12 - ES - EKF',
 'U12 - EM - EKF',
 'U13 - ES - EKF',
 'U13 - EM - EKF',
 'P8 - EM - EKF',
 'Q5 - SCADA',
 'U6 - ES - EKF',
 'U6 - EM - EKF',
 'P6 - NI',
 'U11 - ES - EKF',
 'U11 - EM - EKF',
 'P10 - EM - EKF',
 'P13 - NI',
 'U3 - ES - EKF',
 'U3 - EM - EKF',
 'P12 - SCADA',
 'P9 - SCADA',
 'P11 - SCADA',
 'Q13 - NI',
 'Q4 - NI',
 'Q12 - SCADA',
 'Q6 - NI',
 'U4 - ES - EKF',
 'U4 - EM - EKF',
 'U14 - ES - EKF',
 'U14 - EM - EKF',
 'P9 - NI

In [21]:
X_best = X_fdi.loc[:, best_features] 
X_train, X_test, y_train, y_test = data_preparation.split_data(X_best, y_fdi, test_size=0.2, random_state=123)

train = 1

if train==1:
    # training
    train_LR = -time.time()
    lr_train = LogisticRegression(random_state=0)
    lr_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\lr_model_mrmr.pickle', 'wb') as f:
        pickle.dump(lr_train, f)
    train_LR += time.time()
    print('LR')

    train_KNN = -time.time()
    knn_train = KNeighborsClassifier(n_neighbors=3,
                               weights='distance',
                               )
    knn_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\knn_model_mrmr.pickle', 'wb') as f:
        pickle.dump(knn_train, f)
    train_KNN += time.time()
    print('KNN')

    train_RF = -time.time()
    rf_train = RandomForestClassifier(n_estimators=600,
                                max_depth=14,
                                min_samples_leaf=4, 
                                min_samples_split=4,
                                random_state=0,
                                n_jobs=-1,
                                )
    rf_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\rf_model_mrmr.pickle', 'wb') as f:
        pickle.dump(rf_train, f)
    train_RF += time.time()
    print('RF')
    
    train_XGB = -time.time()
    xgb_train = XGBClassifier(n_estimators=400,
                        max_depth=15,
                        eta=0.3,
                        subsample=0.9,
                        colsample_bytree=0.5, 
                        random_state=0,
                        n_jobs= -1,
                        silent=True)
    xgb_train.fit(X_train, y_train)
    with open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\xgb_model_mrmr.pickle', 'wb') as f:
        pickle.dump(xgb_train, f)
    train_XGB += time.time()
    print('XGB')
    
    train_time = data_preparation.to_dict(train_LR, train_KNN, train_RF, train_XGB)
    data_preparation.save_model(train_time, 'time/train_time_mrmr')


train_time = data_preparation.load_model('time/train_time_mrmr')

# load models
lr = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\lr_model_mrmr.pickle', 'rb'))
knn = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\knn_model_mrmr.pickle', 'rb'))
rf = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\rf_model_mrmr.pickle', 'rb'))
xgb = pickle.load(open(module_path + '\\multi_attack\\SE_bus_FDI\\classification_models\\xgb_model_mrmr.pickle', 'rb'))


# prediction
test_LR = -time.time()
y_pred_lr = lr.predict(X_test)
test_LR += time.time()

test_KNN = -time.time()
y_pred_knn = knn.predict(X_test)
test_KNN += time.time()

test_RF = -time.time()
y_pred_rf = rf.predict(X_test)
test_RF += time.time()

test_XGB = -time.time()
y_pred_xgb = xgb.predict(X_test)
test_XGB += time.time()

# F1 score
macro_f1_lr = f1_score(y_test, y_pred_lr, average='macro')
macro_f1_knn = f1_score(y_test, y_pred_knn, average='macro')
macro_f1_rf = f1_score(y_test, y_pred_rf, average='macro')
macro_f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

LR
KNN
RF
XGB


In [24]:
print('##############')
print('F1 score:')
print('LR', macro_f1_lr*100) 
print('KNN', macro_f1_knn*100)
print('RF', macro_f1_rf*100)
print('XGB', macro_f1_xgb*100)

print('##############')
print('Training Time:')
print('LR', train_time['LR'], 'sec')
print('KNN', train_time['KNN'], 'sec')
print('RF', train_time['RF'], 'sec')
print('XGB', train_time['XGB'], 'sec')

print('##############')
print('Testing Time:')
print('LR', test_LR, 'sec')
print('KNN', test_KNN, 'sec')
print('RF', test_RF, 'sec')
print('XGB', test_XGB, 'sec')

##############
F1 score:
LR 6.228013984317134
KNN 21.37775236249178
RF 58.2068743272483
XGB 47.92978434369162
##############
Training Time:
LR 15.415916681289673 sec
KNN 0.06250643730163574 sec
RF 58.32768106460571 sec
XGB 540.525060415268 sec
##############
Testing Time:
LR 0.015627384185791016 sec
KNN 0.21874642372131348 sec
RF 2.219407320022583 sec
XGB 38.44492483139038 sec
